In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
import scipy.stats


In [2]:
board_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/turn_board/maintenance_turns_visitors_04-20-21.csv'
details_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/turn_board/maintenance_turns_details_visitors_04-20-21.csv'
user_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/turn_board/turn_users_04-20-21.csv'
experiment_group_file = '/Users/bradley.webb/Desktop/mid_market_measurement/data/turn_board/experiment_group_assignment.csv'

In [3]:
df_board = pd.read_csv(board_file)
df_details = pd.read_csv(details_file)
df_users = pd.read_csv(user_file)
df_experiment = pd.read_csv(experiment_group_file)

In [4]:
df_users.head()

,ID,VHOST,EMAIL,UUID_PERMISSIONS_TABLE,UUID_USER_TABLE
0,157,fcrealtors,krobinson@fcrealtors.com,611eb331-7b17-11eb-beca-12ce6fac6131,15de08c9-0ebe-11e9-aea7-b083fedec91d
1,19,kading,karie@kadingproperties.com,a54b07bc-7b06-11eb-9c0b-0a1d416c3178,4c47600f-ebcf-11ea-a8d4-0a5a45d3dd18
2,21,kading,heather@kadingproperties.com,b707e47f-7b06-11eb-9c0b-0a1d416c3178,7afafa76-ebcf-11ea-a8d4-0a5a45d3dd18
3,23,kading,susanpfeilconsulting@gmail.com,a9c88b9c-7b06-11eb-9c0b-0a1d416c3178,0b93ec2e-ec6d-11ea-a8d4-0a5a45d3dd18
4,25,kading,lindsey@kadingproperties.com,a177594e-7b06-11eb-9c0b-0a1d416c3178,b33b2c42-f779-11ea-a8d4-0a5a45d3dd18


In [7]:
df_details.columns

Index(['Visitor ID', 'Page Views for Maintenance > Unit Turn Details',
       'Days Active for Maintenance > Unit Turn Details',
       'Time On Page (minutes) for Maintenance > Unit Turn Details', 'name',
       'email'],
      dtype='object')

In [19]:
# df_details['email'].unique().shape[0]
# df_details[['name', 'email']].value_counts()
# df_details['VHOST_EMAIL'] = df_details.apply(lambda row: row['name'] + '-' + row['email'], axis=1)
# int_df = pd.merge(d1, d2, how ='inner', on =['A', 'B'])
# df_users.join(df_details.set_index('email'), on='EMAIL', rsuffix='_details')
# df_concat = pd.concat([pd.merge(df_users, df_details, how='inner', left_on=['UUID_USER_TABLE'], right_on=['VISITOR_ID']),
# pd.merge(df_users, df_details, how='inner', left_on=['UUID_USER_TABLE'], right_on=['Visitor ID'])
# pd.merge(df_users, df_details, how='inner', left_on=['EMAIL'], right_on=['email'])])


0                sustainrealty-jrossi@sustainrealty.com
1                           wangard-mdomres@wangard.com
2           collinspropmgmt-ccollins@collinspmgroup.com
3                    regencypm-npatino@regencyprops.com
4                   kading-myranda@kadingproperties.com
5             viewpointequitiesinc-maria@vpequities.com
6                        rentahome-sam@gsorentahome.com
7                         rec-reception@rec-rentals.com
8              atlasmanagementor-jakeg@atlasmgmtpdx.com
9               phillipsrealestate-lsnee@phillipsre.com
10                    augusta-maria@augusta-abilene.com
11              laureatetld-jamiebelitz@laureateltd.com
12             skypm-trevor@bgskypropertymanagement.com
13              realpmredding-shastaview@rpmredding.com
14    butlerpropertycompany-andrew@butlerpropertycom...
15                     schwalb-jamesd@schwalbrealty.com
16                    northpointe-ashley.nppm@gmail.com
17                         meccamgmt-lsuter@mecc

In [44]:
df_details_users = pd.merge(df_users, df_details, how='inner', left_on=['EMAIL', 'VHOST'], right_on=['email', 'name'])
df_details_users = pd.merge(df_details_users, df_experiment, how='left', left_on=['VHOST'], right_on=['VHOST'])
df_details_users['GROUP'].fillna(value='control', inplace=True)
df_details_users['GROUP'].unique()

array(['control', 'DIY-onboarding', 'no-onboarding'], dtype=object)

In [50]:
for val in df_details_users['GROUP'].unique():
    df_details_users[val+'_categorical'] = df_details_users['GROUP'].apply(lambda row: 1 if row == val else 0)
df_details_users.head()
# df[df.groupby("sym")["sym"].transform('size') == 2]
# df_users[df_users.groupby('UUID')['UUID'].transform('size') > 1].shape[0]

,ID,VHOST,EMAIL,UUID_PERMISSIONS_TABLE,UUID_USER_TABLE,Visitor ID,Page Views for Maintenance > Unit Turn Details,Days Active for Maintenance > Unit Turn Details,Time On Page (minutes) for Maintenance > Unit Turn Details,name,email,GROUP,control,DIY-onboarding,no-onboarding,control_categorical,DIY-onboarding_categorical,no-onboarding_categorical
0,157,fcrealtors,krobinson@fcrealtors.com,611eb331-7b17-11eb-beca-12ce6fac6131,15de08c9-0ebe-11e9-aea7-b083fedec91d,15de08c9-0ebe-11e9-aea7-b083fedec91d,1,1,2,fcrealtors,krobinson@fcrealtors.com,control,1,0,0,1,0,0
1,21,kading,heather@kadingproperties.com,b707e47f-7b06-11eb-9c0b-0a1d416c3178,7afafa76-ebcf-11ea-a8d4-0a5a45d3dd18,7afafa76-ebcf-11ea-a8d4-0a5a45d3dd18,1,1,2,kading,heather@kadingproperties.com,control,1,0,0,1,0,0
2,23,kading,susanpfeilconsulting@gmail.com,a9c88b9c-7b06-11eb-9c0b-0a1d416c3178,0b93ec2e-ec6d-11ea-a8d4-0a5a45d3dd18,0b93ec2e-ec6d-11ea-a8d4-0a5a45d3dd18,49,6,74,kading,susanpfeilconsulting@gmail.com,control,1,0,0,1,0,0
3,25,kading,lindsey@kadingproperties.com,a177594e-7b06-11eb-9c0b-0a1d416c3178,b33b2c42-f779-11ea-a8d4-0a5a45d3dd18,b33b2c42-f779-11ea-a8d4-0a5a45d3dd18,3,3,4,kading,lindsey@kadingproperties.com,control,1,0,0,1,0,0
4,26,kading,tyler@kadingproperties.com,ace79e2b-7b06-11eb-9c0b-0a1d416c3178,c3564f29-f779-11ea-a8d4-0a5a45d3dd18,c3564f29-f779-11ea-a8d4-0a5a45d3dd18,2,2,2,kading,tyler@kadingproperties.com,control,1,0,0,1,0,0


In [60]:
df_details_users.rename(columns={'Page Views for Maintenance > Unit Turn Details':'PAGE_VIEWS',\
                         'Days Active for Maintenance > Unit Turn Details': 'DAYS_ACTIVE',\
                         'Time On Page (minutes) for Maintenance > Unit Turn Details': 'TIME_SPENT'
                         }, inplace=True)

In [88]:
x = df_details_users[['TIME_SPENT']].to_numpy()
y = df_details_users[['control_categorical']].to_numpy()

In [91]:
model = LinearRegression().fit(x,y)
r_sq = model.score(x,y)
pd.set_option('display.max_columns', 500)
print(df_details_users[['GROUP','PAGE_VIEWS', 'TIME_SPENT']].groupby('GROUP').agg(['count', 'mean', 'median', 'std']))
print('coefficient of determination', r_sq)

               PAGE_VIEWS                              TIME_SPENT             \
                    count       mean median        std      count       mean   
GROUP                                                                          
DIY-onboarding         43  16.627907    3.0  42.976751         43  26.372093   
control              1533  16.091977    4.0  42.268553       1533  26.200913   
no-onboarding          78  42.692308    5.5  94.785925         78  56.923077   

                                   
               median         std  
GROUP                              
DIY-onboarding      5   56.944009  
control             6   84.803082  
no-onboarding       8  115.995048  
coefficient of determination 0.0036098906987819257


In [114]:
x = df_details_users[df_details_users['GROUP'] == 'no-onboarding'].mean()['PAGE_VIEWS']
mu = df_details_users[df_details_users['GROUP'] == 'control'].mean()['PAGE_VIEWS']
std = df_details_users[df_details_users['GROUP'] == 'control'].std()['PAGE_VIEWS']
z = (x - mu)/std
z

crit_val = scipy.stats.norm.ppf(1-.05/2)
print(z, crit_val, z > crit_val)

0.6293172895047672 1.959963984540054 False
